# E01

I did not get around to seeing what happens when you initialize all weights and biases to zero. Try this and train the neural net. You might think either that 1) the network trains just fine or 2) the network doesn't train at all, but actually it is 3) the network trains but only partially, and achieves a pretty bad final performance. 

Inspect the gradients and activations to figure out what is happening and why the network is only partially training, and what part is being trained exactly.


I used this structure:


    layers = [
        Linear(n_embd * block_size, n_hidden), Tanh(),
        Linear(n_hidden,            n_hidden), Tanh(),
        Linear(n_hidden,            n_hidden), Tanh(),
        Linear(n_hidden,            n_hidden), Tanh(),
        Linear(n_hidden,            n_hidden), Tanh(),
        Linear(n_hidden, vocab_size), 
    ]

where all weights and biases of all linear layers are set to 0 during initialization. This means, the output of any linear layer is just 0, including the last one. Tanh sets any 0 input to 0. Hence, the last layer always receives 0 as input. The form of the model is therefore

    softmax(W*0 + b)

and the biases of the last layer are the only parameters being trained

# E02

BatchNorm, unlike other normalization layers like LayerNorm/GroupNorm etc. has the big advantage that after training, the batchnorm gamma/beta can be "folded into" the weights of the preceeding Linear layers, effectively erasing the need to forward it at test time. Set up a small 3-layer MLP with batchnorms, train the network, then "fold" the batchnorm gamma/beta into the preceeding Linear layer's W,b by creating a new W2, b2 and erasing the batch norm. Verify that this gives the same forward pass during inference. i.e. we see that the batchnorm is there just for stabilizing the training, and can be thrown out after training is done! pretty cool.

We have the following pseudo-calculation:

    hpreact = embcat @ W1 + b1
    -> hpreact = bngain * (hpreact - bnmean) / bnstd + bnbias
    -> hpreact = bngain * (embcat @ W1 + b1 - bnmean) / bnstd + bnbias
    -> hpreact = (embcat @ (bngain * W1) / bnstd + ((bngain * b1) - (bngain * bnmean)) / bnstd )+ bnbias
    -> hpreact = embcat @ W2 + b2 

where

    W2 = (bngain * W1) / bnstd 
    b2 = ((bngain * b1) - (bngain * bnmean)) / bnstd) + bnbias

the dimensions are:
* bngain, bnstd, bnbias: n_hidden
* W1: (n_embd * block_size, n_hidden) or (n_hidden, n_hidden) 

# E03 (self-imposed)

Read these papers:
Useful links:
- "Kaiming init" paper: https://arxiv.org/abs/1502.01852
- "PReLu" where the part < 0 is for example -0.1*x, but the -0.1 is a free parameter
- BatchNorm paper: https://arxiv.org/abs/1502.03167
- Good paper illustrating some of the problems with batchnorm in practice: https://arxiv.org/abs/2105.07576
- Write a comment on the iterative updating of running mean and standard deviation. Convergence properties of moving exponential averages are well known, given 'momentum' and distributional properties of the quantity that is supposed to be updated.
- Look at adam optimizer